## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

## Import Packages

In [ ]:
import numpy as np
import cv2
import glob
import os 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

## Calculate camera calibration
This section calculate distortion matrix using chessboard images. multiple chessboard images used to determine distortion matrix

In [ ]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)

objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    
    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    

## Undistort image function
Undistort imgae using calibration matrix

In [ ]:
def undistort_image(distorted_image):
    #Correct distortion using matrix from previous cell
    
    undist = cv2.undistort(distorted_image, mtx, dist, None, mtx)
    return undist

## Process chessboard images for distortion correction

In [ ]:
images1 = glob.glob('./camera_cal/calibration*.jpg')
for fname1 in images1:
    #image output name
    filename=os.path.split(fname1)[-1]
    outfile='./output_images/' + filename +'.jpg'
    
    #read image in pipeline
    img1 = cv2.imread(fname1)
    undist = cv2.undistort(img1, mtx, dist, None, mtx)

    #Process image and save
    '''f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img1)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(undist)
    ax2.set_title('Undistorted Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    f.savefig(outfile)'''

## Image thresholding
I am using s channel for color thresholding and experimenting multiple channels on test images and then evaluate best channel to use gradient. I have decided to combine  s channel color threshold and s-channel x gradient and direction gradient to filter out pixels in horizontal direction

In [ ]:
def get_image_binary(img_rgb,sthresh,sxthresh,gradthresh):
    
    #convert color into HLS
    img_hls = cv2.cvtColor(img_rgb,cv2.COLOR_RGB2HLS)
    img_s=img_hls[:,:,2]
    img_l=img_hls[:,:,1]
    
    #Create binary based on color thresholding
    s_binary = np.zeros_like(img_s)
    s_binary[(img_s >= sthresh[0]) & (img_s <= sthresh[1])]=1
    
    #Gradient thresholding
    #Kernel Size
    sobel_kernel=5
    
    #S-Channel  gradient threshold
    sobel_sx=cv2.Sobel(img_s, cv2.CV_64F, 1, 0,ksize=sobel_kernel)
    sobel_sy=cv2.Sobel(img_s, cv2.CV_64F, 0, 1,ksize=sobel_kernel)
    abs_sobel_sx = np.absolute(sobel_sx) 
    scaled_sobels_sx = np.uint8(255*abs_sobel_sx/np.max(abs_sobel_sx))
    #Create S-channel binary
    sx_binary = np.zeros_like(scaled_sobels_sx)
    sx_binary[(scaled_sobels_sx >= sxthresh[0]) & (scaled_sobels_sx <= sxthresh[1]) ] = 1
    
    #S-Channel direction threshold
    absgraddir = np.arctan2(np.absolute(sobel_sy), np.absolute(sobel_sx))
    binary_output =  np.zeros_like(scaled_sobels_sx)
    binary_output[(absgraddir >= gradthresh[0]) & (absgraddir <= gradthresh[1])] = 1
    
    #Plotting each thresholding
    '''Combined=np.dstack((s_binary,sx_binary,binary_output))*255
    plt.imshow(Combined)
    plt.show()
    cv2.putText(Combined,'S-Channel colour thresholding', (200,50),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),2)
    cv2.putText(Combined,'X direction mag threshold on S-Channel', (200,80),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2) 
    cv2.putText(Combined,'Direction threshold between 0 to PI/3(Red channels)', (200,110),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2) 
    cv2.imwrite('./output_images/gradien_distribution.jpg',Combined)'''
    
    binary=np.zeros_like(sx_binary)
    binary[((s_binary == 1) | (sx_binary == 1)) & (binary_output==1)] =1
    
    return binary

## Transform image
Image perspective transformation to top down view

In [ ]:
def transform_image(src_image,src,dst):
    #Transform imarge to top down view
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(src_image, M, (src_image.shape[1],src_image.shape[0]))
    return warped, M, Minv


## Histogram


In [ ]:
def hist(warp_img):
    # Grab only the bottom half of the image
    # Lane lines are likely to be mostly vertical nearest to the car
    bottom_half = warp_img[warp_img.shape[0]//2:,:]

    # Sum across image pixels vertically - make sure to set an `axis`
    # i.e. the highest areas of vertical lines should be larger values
    histogram = np.sum(bottom_half, axis=0)
    
    return histogram

## Sliding window search
Find lane in frame using sliding window search

In [ ]:
def slidingwindow_search(bin_img,hist):
    #Determine left and right line start place for search
    img_midpoint=hist.shape[0]//2
    left_start_x=np.argmax(hist[:img_midpoint])
    right_start_x=np.argmax(hist[img_midpoint:])+img_midpoint
    
    #Heperparameter
    nwindows=10
    winmargin=100
    minpix=40
    maxpix=7000
    winheight=np.int(bin_img.shape[0]/nwindows)
    
    #List for collecting lane pixels
    left_lane_ids=[]
    right_lane_ids=[]
    
    #Nonzero pixels in image
    bin_img_nonzero=bin_img.nonzero()
    bin_img_nonzeroy=np.array(bin_img_nonzero[0])
    bin_img_nonzerox=np.array(bin_img_nonzero[1])
    
    #Ploting
    out_img = np.dstack((bin_img, bin_img, bin_img))*255
    
    
    
    #looping window
    currentx_l=left_start_x
    currentx_r=right_start_x
    
    #print(currentx_l)
    #print(currentx_r)
    
    for window in range(nwindows):
        win_yl=bin_img.shape[0]-(window+1)*winheight
        win_yh=bin_img.shape[0]-(window)*winheight
        leftwin_xl=currentx_l-winmargin
        leftwin_xh=currentx_l+winmargin
        rightwin_xl=currentx_r-winmargin
        rightwin_xh=currentx_r+winmargin
        
        #ploting
        cv2.rectangle(out_img,(leftwin_xl,win_yl), (leftwin_xh,win_yh),(0,255,0), 2) 
        cv2.rectangle(out_img,(rightwin_xl,win_yl), (rightwin_xh,win_yh),(0,255,0), 2) 
        
        left_good_ids=((bin_img_nonzeroy <= win_yh) & (bin_img_nonzeroy >= win_yl) 
                      & (bin_img_nonzerox >= leftwin_xl) &(bin_img_nonzerox <= leftwin_xh)).nonzero()[0]
        
        
        right_good_ids=((bin_img_nonzeroy <= win_yh) & (bin_img_nonzeroy >= win_yl) 
                      & (bin_img_nonzerox >= rightwin_xl) &(bin_img_nonzerox <= rightwin_xh)).nonzero()[0]
        
        
        #append points
        left_lane_ids.append(left_good_ids)
        right_lane_ids.append(right_good_ids)
        
       
        #check if points are good enough for update box centre
        if (((len(left_good_ids) > minpix)) 
            & ((len(right_good_ids) > minpix) )):
            
            currentx_l=np.int(np.mean(bin_img_nonzerox[left_good_ids]))
            currentx_r=np.int(np.mean(bin_img_nonzerox[right_good_ids]))
            #print("both")
            #print(currentx_l)
            #print(currentx_r)
        elif ((len(left_good_ids) > minpix) ):
            temp_x=np.int(np.mean(bin_img_nonzerox[left_good_ids]))
            delta=temp_x-currentx_l
            currentx_l=temp_x
            currentx_r=currentx_r+delta
            #print("Only left")
            #print(currentx_l)
            #print(currentx_r)
        elif ((len(right_good_ids) > minpix) ):
            temp_y=np.int(np.mean(bin_img_nonzerox[right_good_ids]))
            delta=temp_y-currentx_r
            currentx_l=currentx_l+delta
            currentx_r=temp_y
            #print("Only right")
            #print(currentx_l)
            #print(currentx_r)
            
       
    #plt.imshow(out_img)
    #plt.show()
    
    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_ids = np.concatenate(left_lane_ids)
        right_lane_ids = np.concatenate(right_lane_ids)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass
    
    #Extract pixel positions
    leftx=bin_img_nonzerox[left_lane_ids]
    lefty=bin_img_nonzeroy[left_lane_ids]
    rightx=bin_img_nonzerox[right_lane_ids]
    righty=bin_img_nonzeroy[right_lane_ids]

    return leftx, lefty, rightx, righty
       

## Generate xy for polynimoial plotting

In [ ]:
 def ploty_poly(image_ploty,left_fit_plt,right_fit_plt, leftx_plt, lefty_plt, rightx_plt, righty_plt):
    # Generate x and y values for plotting
    
    ploty = np.linspace(0, image_ploty.shape[0]-1, image_ploty.shape[0] )
    try:
        left_fitx = left_fit_plt[0]*ploty**2 + left_fit_plt[1]*ploty + left_fit_plt[2]
        right_fitx = right_fit_plt[0]*ploty**2 + right_fit_plt[1]*ploty + right_fit_plt[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    #print(leftx_plt)
    #print(rightx_plt)
    poly_img=np.dstack((np.zeros_like(image_ploty),np.zeros_like(image_ploty),image_ploty))
    poly_img[lefty_plt,leftx_plt,]=[255,0,0]
    poly_img[righty_plt,rightx_plt]=[0,0,255]
    #Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    #plt.imshow(poly_img)
    #plt.show()
    '''fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(poly_img)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    fig.savefig('./output_images/fitted_polyline.jpg')
    '''    
    
    return left_fitx, right_fitx, ploty

## Polynomial fit
Fiting pixels into second order polynomial

In [ ]:
def fit_polynomial(input_image,leftx, lefty, rightx, righty):
    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    left_fitx,right_fitx, ploty =ploty_poly(input_image,left_fit,right_fit, leftx, lefty, rightx, righty)
       
        
    return left_fit, right_fit, left_fitx, right_fitx

    

## Search the lane using polynomial in previous frame 

In [ ]:
def search_around_poly(binary_warped,prev_left_fit,prev_right_fit):
    # HYPERPARAMETER
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    left_lane_inds_ = ((nonzerox > (prev_left_fit[0]*(nonzeroy**2) + prev_left_fit[1]*nonzeroy + 
                    prev_left_fit[2] - margin)) & (nonzerox < (prev_left_fit[0]*(nonzeroy**2) + 
                    prev_left_fit[1]*nonzeroy + prev_left_fit[2] + margin)))
    right_lane_inds_ = ((nonzerox > (prev_right_fit[0]*(nonzeroy**2) + prev_right_fit[1]*nonzeroy + 
                    prev_right_fit[2] - margin)) & (nonzerox < (prev_right_fit[0]*(nonzeroy**2) + 
                    prev_right_fit[1]*nonzeroy + prev_right_fit[2] + margin)))
    
    # extract left and right line pixel positions
    leftx_polysearch = nonzerox[left_lane_inds_]
    lefty_polysearch = nonzeroy[left_lane_inds_] 
    rightx_polysearch = nonzerox[right_lane_inds_]
    righty_polysearch = nonzeroy[right_lane_inds_]
    
    if((len(leftx_polysearch) > 2) & (len(lefty_polysearch) > 2) & (len(rightx_polysearch) > 2) & (len(righty_polysearch) > 2)):
        
        # Fit new polynomials
        left_fit_, right_fit_ , left_fitx, right_fitx = fit_polynomial(binary_warped, leftx_polysearch, lefty_polysearch, rightx_polysearch, righty_polysearch)


        left_fiteedx, right_fittedx, ploty_ =ploty_poly(binary_warped,left_fit_,right_fit_,\
                                                 leftx_polysearch, lefty_polysearch, rightx_polysearch, righty_polysearch)
    else:
        left_fit_=prev_left_fit
        right_fit_=prev_right_fit
        left_fiteedx=[]
        right_fittedx=[]

    return left_fit_, right_fit_, left_fiteedx, right_fittedx, leftx_polysearch, lefty_polysearch, rightx_polysearch, righty_polysearch

## Convert pixel polynomial into real world polynomial
Convert polynomial into ral world polynomial equation

In [ ]:
def convert_poly2realworld(leftfit_pix,right_fit_pix,x_pix2realworl, y_pix2realworld):
    leftfit_real=[0 for x in range(3)]
    rightfit_real=[0 for x in range(3)]
    
    leftfit_real[0]=(leftfit_pix[0]*x_pix2realworl)/(y_pix2realworld**2)
    leftfit_real[1]=(leftfit_pix[1]*x_pix2realworl)/(y_pix2realworld)
    leftfit_real[2]=(leftfit_pix[2]*x_pix2realworl)
    
    rightfit_real[0]=(right_fit_pix[0]*x_pix2realworl)/(y_pix2realworld**2)
    rightfit_real[1]=(right_fit_pix[1]*x_pix2realworl)/(y_pix2realworld)
    rightfit_real[2]=(right_fit_pix[2]*x_pix2realworl)
    
    return leftfit_real, rightfit_real

## Measure radius of curvature 

In [ ]:
def meas_curvature(left_fitpoly,right_fitply,y_instance):
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fitpoly[0]*y_instance + left_fitpoly[1])**2)**1.5) / np.absolute(2*left_fitpoly[0])
    right_curverad = ((1 + (2*right_fitply[0]*y_instance + right_fitply[1])**2)**1.5) / np.absolute(2*right_fitply[0])
    
    return left_curverad,right_curverad
    
    

## Lane drawn on road

In [ ]:
def draw_image(src_img,im_warped,im_left_fitx,im_right_fitx,im_ploty,Minv,text1,text2):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(im_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([im_left_fitx, im_ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([im_right_fitx, im_ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (im_warped.shape[1], im_warped.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(src_img, 1, newwarp, 0.3, 0)
    
    cv2.putText(result,text1, (200,50),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2)
    cv2.putText(result,text2, (200,80),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2)            
    #plt.imshow(result)
    #plt.show()
    
    return result

## Line class

In [ ]:
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = np.array([0,0,0], dtype='float')   
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx =[]
        #y values for detected line pixels
        self.ally = []
        #number of frame missed
        self.missed=0

## Lane objects

In [ ]:
left_lane=Line()
right_lane=Line()

## Process image
Crearte a fiunction to process image in pipeline

In [ ]:
def process_image(movie_img):
    #Covert image into BGR
    movie_rgb= cv2.cvtColor(movie_img,cv2.COLOR_BGR2RGB)
    #plt.imshow(movie_rgb)
    #plt.show()
    
    #Distortion correction
    undist_img=undistort_image(movie_rgb)
    
    
    #Binary image after gradient and color thresholding
    colr_thres=(140,255)
    grad_mag_thres=(20,100)
    grad_dir_thres=(0,np.pi/3)
    grad_img=get_image_binary(undist_img,colr_thres,grad_mag_thres,grad_dir_thres)
    #plt.imshow(grad_img)
    #plt.show()
    
    
    #Perspective transformation
    src_corners = np.array([[592,450], [690,450], [1070,700], [245,700]])
    dst_corners = np.array([[200,100], [1000,100], [1000,650], [200,650]])
    
    #Plot region of interest for transformation
    gradout=np.dstack((np.zeros_like(grad_img),np.zeros_like(grad_img),grad_img))*255
    
    cv2.polylines(gradout, [dst_corners], 1, (255,0,0),4)
    #plt.imshow(gradout)
    #plt.show()
    
    
    #image to top down
    warped_img, M_warped, Minv_warpd =transform_image(grad_img,np.float32(src_corners),np.float32(dst_corners))
    
    
    warp_zero = np.zeros_like(warped_img).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warped_img))*255
    #plt.imshow(color_warp)
    #plt.show()
    newwarp = cv2.warpPerspective(color_warp, Minv_warpd, (color_warp.shape[1], color_warp.shape[0])) 
    #plt.imshow(newwarp)
    #plt.show()
    
    
    #top down image plot
    warpout=np.dstack((np.zeros_like(warped_img),np.zeros_like(warped_img),warped_img))*255
    #cv2.rectangle(warpout,(200,100), (1000,650),(255,0,0), 2) 
    #plt.imshow(warpout)
    #plt.show()
    
    
    #Search sliding window if line detected is not true
    if ((left_lane.detected == False) | (right_lane.detected == False)):
        #print('Sliding window search')
        
        #Historgram for finding best place to start
        histogram=hist(warped_img/255)
        
        #Search lane pixelusing sliding window search
        leftx_i, lefty_i, rightx_i, righty_i=slidingwindow_search(warped_img,histogram)
        
             
        #Store detected pixel cordinates for left lane
        left_lane.allx.append(leftx_i)
        left_lane.ally.append(lefty_i)

        #Store detected pixel cordinates for right lane
        right_lane.allx.append(rightx_i)
        right_lane.ally.append(righty_i)
        
        
        last_nxleft=np.concatenate((left_lane.allx[-10:]))
        last_nyleft=np.concatenate((left_lane.ally[-10:]))
        last_nxright=np.concatenate((right_lane.allx[-10:]))
        last_nyright=np.concatenate((right_lane.ally[-10:]))
        
        #Procedd further only if two points atleast detected
        if ((len(last_nxleft) >2) & (len(last_nyleft) >2) &
           (len(last_nxright) >2) & (len(last_nxright) >2)):
            
            #fit polynimial 
            left_lane.current_fit,right_lane.current_fit,left_current_xfitted, right_current_xfitted = fit_polynomial(
                warped_img,last_nxleft, last_nyleft,last_nxright, last_nyright)
            
    else:
        #print('Around Poly search')
        
        #Search lane around the existing polynomial
        left_lane.current_fit,right_lane.current_fit,\
        left_current_xfitted, right_current_xfitted, \
        leftx_i, lefty_i, rightx_i, righty_i = search_around_poly(warped_img,left_lane.best_fit,right_lane.best_fit)
        
        
        #Store detected pixel cordinates for left lane
        left_lane.allx.append(leftx_i)
        left_lane.ally.append(lefty_i)
                
        #Store detected pixel cordinates for right lane
        right_lane.allx.append(rightx_i)
        right_lane.ally.append(righty_i)
        
    
      
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 35/640 # meters per pixel in y dimension
    xm_per_pix = 3.7/850 # meters per pixel in x dimension
    left_fit_real,right_fit_real = convert_poly2realworld(left_lane.current_fit,right_lane.current_fit,xm_per_pix, ym_per_pix)
    
    #Measure curvature at y=700
    left_curv,right_curv=meas_curvature(left_fit_real,right_fit_real,700)
    
    #print('leftcur',left_curv,'right cur',right_curv)
    #Sanity check of detcted line
    # if mean difference in fited x are more than 50 pixel then line may not beparalle
    # if curvature is too high then fitting is wrong
    #if both lane curvature differing too much then fitting is wrong
    #print((np.mean(right_current_xfitted-(left_current_xfitted+800))))
    #print(np.maximum(left_curv,right_curv))
    #print(np.abs(left_curv-right_curv))
    
    if ((len(right_current_xfitted)==0) | (len(left_current_xfitted)==0)):
        #increase missed frame count
        left_lane.missed=left_lane.missed+1
        right_lane.missed=right_lane.missed+1
        
        out_image=movie_img
        
    elif (np.abs(np.mean(right_current_xfitted-(left_current_xfitted+800)) > 100)\
    | (np.maximum(left_curv,right_curv) > 15000.)  | (np.abs(left_curv-right_curv) > 12000.)):
        #print('Lane Missed')
        
        #increase missed frame count
        left_lane.missed=left_lane.missed+1
        right_lane.missed=right_lane.missed+1
        
        out_image=movie_img
    else:
        #print('Lane found')
        
        #reset missed frame count
        left_lane.missed=0
        right_lane.missed=0
        
        #Set lane detected
        left_lane.detected=True
        right_lane.detected=True
        
         # x values of the last n fits of the line
        left_lane.recent_xfitted.append(left_current_xfitted)
        right_lane.recent_xfitted.append(right_current_xfitted)
        
        
        #store only 10 values
        if(len(left_lane.recent_xfitted)>10):
            left_lane.recent_xfitted.pop(0)
        if(len(right_lane.recent_xfitted)>10):
            right_lane.recent_xfitted.pop(0)
        
        
        
        #average x values of the fitted line over the last n iterations
        left_lane.bestx = np.mean([left_lane.recent_xfitted], axis=0)
        right_lane.bestx = np.mean([right_lane.recent_xfitted], axis=0)
        
               
        #difference in fit coefficients between last and new fits
        left_lane.diffs = left_lane.best_fit - left_lane.current_fit
        right_lane.diffs =right_lane.best_fit - right_lane.current_fit
        
        #polynomial coefficients averaged over the last n iterations
        if (all(left_lane.best_fit == 0)):
            left_lane.best_fit = left_lane.current_fit
        else:
            left_lane.best_fit = (left_lane.best_fit + left_lane.current_fit)/2
        
        if (all(right_lane.best_fit == 0)):
            right_lane.best_fit = right_lane.current_fit
        else:
            right_lane.best_fit = (right_lane.best_fit + right_lane.current_fit)/2
                
                                               
        #radius of curvature of the line in some units
        left_lane.radius_of_curvature = left_curv 
        right_lane.radius_of_curvature = right_curv 
        
        
        
        #print(left_lane.line_base_pos)
        #print(right_lane.line_base_pos)
        last_nxright=np.concatenate((right_lane.allx[-10:]))
        last_nyright=np.concatenate((right_lane.ally[-10:]))
        last_nxleft=np.concatenate((left_lane.allx[-10:]))
        last_nyleft=np.concatenate((left_lane.ally[-10:]))
        
        xfitted_left, xfitted_right,ploty_current=ploty_poly(
            warped_img,left_lane.best_fit ,right_lane.best_fit,last_nxleft,last_nyleft,last_nxright,last_nyright)
        
        #print(movie_img.shape[1]/2)
        #print(xfitted_left[-1])
        #print(xfitted_right[-1])
        #distance in meters of vehicle center from the line
        left_lane.line_base_pos= ((movie_img.shape[1]/2)-xfitted_left[-1])* xm_per_pix 
        right_lane.line_base_pos= (xfitted_right[-1] -(movie_img.shape[1]/2))* xm_per_pix
        #print(left_lane.line_base_pos)
        #print(right_lane.line_base_pos)
        
        #Print Text
        text_radcurv='Radius of Curvature = ' +str(round(np.minimum(left_curv,right_curv),1)) + '(m)'
        if (left_lane.line_base_pos == right_lane.line_base_pos ):
            text_centr='vehicle is in center of lane'
        elif(left_lane.line_base_pos < right_lane.line_base_pos ):
            text_centr='vehicle is '+ str(round((right_lane.line_base_pos- left_lane.line_base_pos),2)) +'m left of center'
        else:
            text_centr='vehicle is '+str(round((left_lane.line_base_pos - right_lane.line_base_pos),2)) +'m right of center'
        
        #plt.imshow(warped_img)
        #plt.show()
        out_image = draw_image(movie_img,warped_img,xfitted_left,xfitted_right,ploty_current,Minv_warpd,text_radcurv, text_centr)
        
        
    if ((left_lane.missed >=5 ) | (right_lane.missed >=5 )):
        left_lane.detected=False
        right_lane.detected=False
    
    return out_image

## Test image pipeline

In [ ]:
# Make a list of calibration images
images = glob.glob('./test_images/*.jpg')
print(images)
for fim in images:
    print(fim)
    left_lane=Line()
    right_lane=Line()
    curimage=cv2.imread(images[6])
  
    x = process_image(curimage)    


In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
test_output = './output_video/project_video1.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("project_video.mp4").subclip(0,0.02)
clip1 = VideoFileClip("project_video.mp4")
left_lane=Line()
right_lane=Line()
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(test_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(test_output))

## Challenge video

In [ ]:
test_output = './output_video/challenge_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("project_video.mp4").subclip(0,1)
clip1 = VideoFileClip("challenge_video.mp4")
left_lane=Line()
right_lane=Line()
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(test_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(test_output))

## Harder Challenge

In [ ]:
test_output = './output_video/harder_challenge_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("project_video.mp4").subclip(0,1)
clip1 = VideoFileClip("harder_challenge_video.mp4")
left_lane=Line()
right_lane=Line()
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(test_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(test_output))